# Quality Control
This is the first noebook of a serie of three:
- 01_QualityControl: invalid entries of the data are replaced by NA 
- 02_MissingData_complete: the NA entries are filled depending on the chosen method can be analysed in 
- 03_Analysis: The input variables are analysed, the remaining NA values are deleted, further information from the sites is added and machine Learning is used to analyse the data and train a model to allow conclusions about soil moisture based on remotely sensing data

#### This notebook uses:
- data from the satellite products MOD09GA, MODOCGA, MOD11A1 and MYD11A1

The data is publicly available satellite data from Aqua and Terra. It is available through google earth engine and ingestr (https://github.com/computationales/ingestr). A file with all the needed data is also in the folder data (./data/df_lst_and_refl.csv) or can be found on Euler.
#### This notebook contains:
- the quality control for each product
- the replacement of values with NA in case of a failed quality control
- the export of a csv file containing all relevant information and which of the entries passed the quality check (./data/FLUXNET_MODOCGA_MOD09GA_MYD11A1_MOD11A1.csv)

In [1]:
# enables the use of R code in a cell after writing "%%R making it an cell for R code 
%load_ext rpy2.ipython   

<span style="color:red">**To do:**</span> Adapt working directory and download packages. To use the following packages they have to be installed first. Here are listed more packages than necessary since they will be needed later for the analysis: 

`install.packages(c("dplyr","lubridate","readr","ggplot2","tidyr","caret","tidyverse","yardstick","recipes","devtools","MODISTools","binaryLogic","patchwork","LSD","ggthemes","RColorBrewer"))`

To install necessary dependencies for Debian based OS it might help to install the following things first (not within R):

`sudo apt install build-essential libcurl4-gnutls-dev libxml2-dev libssl-dev libcurl4-openssl-dev` 

` sudo apt-get install -y libxml2-dev libcurl4-openssl-dev libssl-dev libproj` 

` sudo apt-get install libmysql++-dev` 


In [2]:
%%capture
%%R 
# set working directory
setwd("/home/zhud/wsl/MasterThesis/rsvi")

source("remove_outliers.R")     # code from predecessor

library(dplyr)                 
library(lubridate)              
library(readr)
library(ggplot2)
library(tidyr)                  
library(tidyverse) 
library(binaryLogic)

Failed to create bus connection: No such file or directory


In [3]:
%%R
# check if the output of this file was already created
dataFile <- "./data/FLUXNET_MODOCGA_MOD09GA1km_2000_2018_bands_processed.csv"
file.exists(dataFile)

[1] FALSE


In [4]:
%%R
# read in the data (it should contain the rows sur_refl_b01, ..., sur_refl_b16, 
#                                              LST_Day_1km_aqua, LST_Day_1km_terra, 
#                                              QC_Day_terra, QC_Day_aqua, QC_500m, QC_b8_15_1km, QC_b16_1km 
#                                              date, id, DD, MM, YY
data = read_csv('./data/df_lst_and_refl.csv')
data

Rows: 241826 Columns: 34
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (2): id, sites
dbl (32): date, year_dec_x, LST_Day_1km_terra, QC_Day_terra, year_dec_y, LST...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
# A tibble: 241,826 × 34
    date year_dec_x id         LST_Day_1km_terra QC_Day_terra sites  year_dec_y
   <dbl>      <dbl> <chr>                  <dbl>        <dbl> <chr>       <dbl>
 1 14245      2009  2009_01_01              307.           65 AR-Vir      2009 
 2 14246      2009. 2009_01_02               NA             2 AR-Vir      2009.
 3 14247      2009. 2009_01_03               NA             2 AR-Vir      2009.
 4 14248      2009. 2009_01_04              303.           65 AR-Vir      2009.
 5 14249      2009. 2009_01_05              307.            0 AR-Vir      2009.
 6 14250      2009. 2009_01_06         

## MOD09GA

In [5]:
%%R
mod09 <- data %>% 
    dplyr::select(YY, MM, DD, sites, QC_500m, state_1km,                                                   ################ state_1km MODOCGA
                  sur_refl_b01, sur_refl_b02, sur_refl_b03, sur_refl_b04, 
                  sur_refl_b05, sur_refl_b06, sur_refl_b07, 
                  ) %>% 
    mutate(date = ymd(paste(as.character(YY), as.character(MM), as.character(DD), sep="-")))
qflags_500 <- sort(unique(mod09$QC_500m)) 

In [6]:
# %%R
# dates_mod09  <- mod09 %>% 
#     mutate(date = ymd(paste(as.character(YY), as.character(MM), as.character(DD), sep="-")))

In [7]:
%%R
# create a dataframe in which the flags are read into
QC_Data_500 <- data.frame(Integer_Value = qflags_500,  
                        Bit31 = NA, 
                        Bit30 = NA, Bit29 = NA, Bit28 = NA, Bit27 = NA, Bit26 = NA, Bit25 = NA, Bit24 = NA, Bit23 = NA, Bit22 = NA, Bit21 = NA, 
                        Bit20 = NA, Bit19 = NA, Bit18 = NA, Bit17 = NA, Bit16 = NA, Bit15 = NA, Bit14 = NA, Bit13 = NA, Bit12 = NA, Bit11 = NA,
                        Bit10 = NA, Bit9 = NA,  Bit8 = NA,  Bit7 = NA,  Bit6 = NA,  Bit5 = NA,  Bit4 = NA,  Bit3 = NA,  Bit2 = NA,  Bit1 = NA,
                        Bit0 = NA)


In [8]:
%%R
# read the integers of qflags_500 into the relevant columns of QC_Data_500
r <- 0
for (i in QC_Data_500$Integer_Value){
    AsInt <- as.integer(intToBits(i))
    # AsInt <- as.integer(as.logical(as.binary(i, n=32, logic=F)))
    QC_Data_500[r+1,2:33]<- AsInt[32:1] # changed from AsInt[32:1] - otherwise Bit0 would not be the LSB as described in the mod09 User Guide
    r <- r+1
}

In [9]:
%%R
my_data <- as_tibble(QC_Data_500)

# columns with the bits of interest for QC_500 of MOD09A1
# more information about the interpreatiion of the flags of QC_500 can be found here:
# https://lpdaac.usgs.gov/products/mod09gav006/
all_bands <- c(33:32)                # Bit 0 and 1
b1 <- c(31:28)                       # Bit 2 - 5
b2 <- c(27:24)                       # Bit 6 - 9
b3 <- c(23:20)                       # Bit 10 - 13
b4 <- c(19:16)                       # Bit 14 - 17
b5 <- c(15:12)                       # Bit 18 - 21 
b6 <- c(11:8)                        # Bit 22 - 25 
b7 <- c(7:4)                         # Bit 26 - 29 
atmospheric_correction <- c(3:3)     # Bit 30
adjacency_correction <- c(2:2)       # Bit 31

# select valid QC_500 values depending on the bands
# selects the first column (select(1) if all Bits are 0 filter_at(vars(-Integer_Value), all_vars(.== 0), of the selected Bits b1 (select(1,b1) --> Bit 2 to 5) of the data (my_data)
# select(my_data, all_bands)
filter_qflags_qc_all_bands <- my_data %>% select(1,all_bands) %>% filter_at(vars(-Integer_Value), all_vars(.== 0)) %>% select(1)
filter_qflags_qc_b1 <- my_data %>% select(1,b1) %>% filter_at(vars(-Integer_Value), all_vars(.== 0)) %>% select(1)
filter_qflags_qc_b2 <- my_data %>% select(1,b2) %>% filter_at(vars(-Integer_Value), all_vars(.== 0)) %>% select(1)
filter_qflags_qc_b3 <- my_data %>% select(1,b3) %>% filter_at(vars(-Integer_Value), all_vars(.== 0)) %>% select(1)
filter_qflags_qc_b4 <- my_data %>% select(1,b4) %>% filter_at(vars(-Integer_Value), all_vars(.== 0)) %>% select(1)
filter_qflags_qc_b5 <- my_data %>% select(1,b5) %>% filter_at(vars(-Integer_Value), all_vars(.== 0)) %>% select(1)
filter_qflags_qc_b6 <- my_data %>% select(1,b6) %>% filter_at(vars(-Integer_Value), all_vars(.== 0)) %>% select(1)
filter_qflags_qc_b7 <- my_data %>% select(1,b7) %>% filter_at(vars(-Integer_Value), all_vars(.== 0)) %>% select(1)
filter_qflags_qc_atmospheric_correction <- my_data %>% select(1,atmospheric_correction) %>% filter_at(vars(-Integer_Value), all_vars(.== 1)) %>% select(1)
filter_qflags_qc_adjacency_correction <- my_data %>% select(1,adjacency_correction) %>% filter_at(vars(-Integer_Value), all_vars(.== 1)) %>% select(1)

Note: Using an external vector in selections is ambiguous.
ℹ Use `all_of(all_bands)` instead of `all_bands` to silence this message.
ℹ See <https://tidyselect.r-lib.org/reference/faq-external-vector.html>.
This message is displayed once per session.
Note: Using an external vector in selections is ambiguous.
ℹ Use `all_of(b1)` instead of `b1` to silence this message.
ℹ See <https://tidyselect.r-lib.org/reference/faq-external-vector.html>.
This message is displayed once per session.
Note: Using an external vector in selections is ambiguous.
ℹ Use `all_of(b2)` instead of `b2` to silence this message.
ℹ See <https://tidyselect.r-lib.org/reference/faq-external-vector.html>.
This message is displayed once per session.
Note: Using an external vector in selections is ambiguous.
ℹ Use `all_of(b3)` instead of `b3` to silence this message.
ℹ See <https://tidyselect.r-lib.org/reference/faq-external-vector.html>.
This message is displayed once per session.
Note: Using an external vector in selectio

In [10]:
%%R
#  copy data into new columns and replace it with NA if quality check failed
mod09$b01 <- mod09$sur_refl_b01 
mod09$b01[!mod09$QC_500m %in% filter_qflags_qc_b1$Integer_Value] <- NA
mod09$b02 <- mod09$sur_refl_b02
mod09$b01[!mod09$QC_500m %in% filter_qflags_qc_b1$Integer_Value] <- NA
mod09$b03 <- mod09$sur_refl_b03 
mod09$b03[!mod09$QC_500m %in% filter_qflags_qc_b3$Integer_Value] <- NA
mod09$b04 <- mod09$sur_refl_b04
mod09$b04[!mod09$QC_500m %in% filter_qflags_qc_b4$Integer_Value] <- NA
mod09$b05 <- mod09$sur_refl_b05
mod09$b05[!mod09$QC_500m %in% filter_qflags_qc_b5$Integer_Value] <- NA
mod09$b06 <- mod09$sur_refl_b06 
mod09$b06[!mod09$QC_500m %in% filter_qflags_qc_b6$Integer_Value] <- NA
mod09$b07 <- mod09$sur_refl_b07
mod09$b07[!mod09$QC_500m %in% filter_qflags_qc_b7$Integer_Value] <- NA

In [11]:
%%R
# adds a flag column if quality check is passed/if corrected (atmospheric and adjacency correction)   
mod09$qc_flag_all_bands<- isTRUE(mod09$QC_500m %in% filter_qflags_qc_all_bands$Integer_Value)
mod09$qc_flag_b01 <- mod09$QC_500m %in% filter_qflags_qc_b1$Integer_Value
mod09$qc_flag_b02 <- mod09$QC_500m %in% filter_qflags_qc_b2$Integer_Value
mod09$qc_flag_b03 <- mod09$QC_500m %in% filter_qflags_qc_b3$Integer_Value
mod09$qc_flag_b04 <- mod09$QC_500m %in% filter_qflags_qc_b4$Integer_Value
mod09$qc_flag_b05 <- mod09$QC_500m %in% filter_qflags_qc_b5$Integer_Value
mod09$qc_flag_b06 <- mod09$QC_500m %in% filter_qflags_qc_b6$Integer_Value
mod09$qc_flag_b07 <- mod09$QC_500m %in% filter_qflags_qc_b7$Integer_Value
mod09$qc_flag_atmospheric_correction<- mod09$QC_500m %in% filter_qflags_qc_atmospheric_correction$Integer_Value
mod09$qc_flag_adjacency_correction<- mod09$QC_500m %in% filter_qflags_qc_adjacency_correction$Integer_Value

## MODOCGA

In [12]:
%%R
modocga <- data %>% 
    dplyr::select(YY, MM, DD, sites, QC_b8_15_1km, QC_b16_1km, 
                  sur_refl_b08, sur_refl_b09, sur_refl_b10, 
                  sur_refl_b11, sur_refl_b12, sur_refl_b13, 
                  sur_refl_b14, sur_refl_b15, sur_refl_b16) %>% 
    mutate(date = ymd(paste(as.character(YY), as.character(MM), as.character(DD), sep="-")))

In [13]:
%%R
# information on the quality control flags can be found here:
# https://ladsweb.modaps.eosdis.nasa.gov/filespec/MODIS/6/MODOCGA
qflags_modocga_b8_to_b15 <- sort(unique(modocga$QC_b8_15_1km))# 32 bits
qflags_modocga_b16       <- sort(unique(modocga$QC_b16_1km))# 8 bits
qflags_state             <- sort(unique(modocga$QC_b16_1km))# 16 bit

In [14]:
%%R
# create dataframes in which the flags are read into
QC_Data_MODOCGA_b8_b15<- data.frame(Integer_Value = qflags_modocga_b8_to_b15, 
                        Bit31 = NA, 
                        Bit30 = NA, Bit29 = NA, Bit28 = NA, Bit27 = NA, Bit26 = NA, Bit25 = NA, Bit24 = NA, Bit23 = NA, Bit22 = NA, Bit21 = NA, 
                        Bit20 = NA, Bit19 = NA, Bit18 = NA, Bit17 = NA, Bit16 = NA, Bit15 = NA, Bit14 = NA, Bit13 = NA, Bit12 = NA, Bit11 = NA,
                        Bit10 = NA, Bit9 = NA,  Bit8 = NA,  Bit7 = NA,  Bit6 = NA,  Bit5 = NA,  Bit4 = NA,  Bit3 = NA,  Bit2 = NA,  Bit1 = NA,
                        Bit0 = NA)
QC_Data_MODOCGA_b16<- data.frame(Integer_Value = qflags_modocga_b16, 
                        Bit7 = NA,  Bit6 = NA,  Bit5 = NA,  Bit4 = NA,  Bit3 = NA,  Bit2 = NA,  Bit1 = NA,
                        Bit0 = NA)
QC_Data_state <- data.frame(Integer_Value = qflags_state, 
                        Bit15 = NA, Bit14 = NA, Bit13 = NA, Bit12 = NA, Bit11 = NA,
                        Bit10 = NA, Bit9 = NA,  Bit8 = NA,  Bit7 = NA,  Bit6 = NA,  Bit5 = NA,  Bit4 = NA,  Bit3 = NA,  Bit2 = NA,  Bit1 = NA,
                        Bit0 = NA)

In [15]:
%%R
# read the integers of the qflags into the relevant columns of QC_Data
r <- 0
for (i in QC_Data_MODOCGA_b8_b15$Integer_Value){
    AsInt <- as.integer(intToBits(i)) # reads in LSB first 
    # AsInt <- as.integer(as.logical(as.binary(i, n=32, logic=F)))
    QC_Data_MODOCGA_b8_b15[r+1,2:33]<- AsInt[32:1] #  AsInt[32:1] - otherwise Bit0 would not be the LSB as described in the mod09 User Guide
    r <- r+1
}
r <- 0
for (i in QC_Data_MODOCGA_b16$Integer_Value){
    AsInt <- as.integer(intToBits(i)) # reads in LSB first 
    # AsInt <- as.integer(as.logical(as.binary(i, n=32, logic=F)))
    QC_Data_MODOCGA_b16[r+1,2:9]<- AsInt[8:1] #  AsInt[32:1] - otherwise Bit0 would not be the LSB as described in the mod09 User Guide
    r <- r+1
}

In [16]:
%%R
my_data_b8_b15 <- as_tibble(QC_Data_MODOCGA_b8_b15)
my_data_b16 <- as_tibble(QC_Data_MODOCGA_b16)
# columns with the bits of interest for QC_1000 of MODOCGA
# https://ladsweb.modaps.eosdis.nasa.gov/filespec/MODIS/6/MODOCGA
b8 <- c(33:30)                       # Bit 0 - 3
b9 <- c(29:26)                       # Bit 4 - 7
b10 <- c(25:22)                      # Bit 8 - 11
b11 <- c(21:18)                      # Bit 12 - 15
b12 <- c(17:14)                      # Bit 16 - 19 
b13 <- c(13:10)                      # Bit 20 - 23 
b14 <- c(9:6)                        # Bit 24 - 27 
b15 <- c(5:2)                        # Bit 28 - 31 
b16 <- c(5:2)                        # Bit 4 - 7

# select valid QC_500 depending on the bands
# selects the first column (select(1) if all Bits are 0 filter_at(vars(-Integer_Value), all_vars(.== 0), of the selected Bits b1 (select(1,b1) --> Bit 2 to 5) of the data (my_data)
# select(my_data, all_bands)
filter_qflags_qc_b08 <- my_data_b8_b15 %>% select(1,b8) %>% filter_at(vars(-Integer_Value), all_vars(.== 0)) %>% select(1)
filter_qflags_qc_b09 <- my_data_b8_b15 %>% select(1,b9) %>% filter_at(vars(-Integer_Value), all_vars(.== 0)) %>% select(1)
filter_qflags_qc_b10 <- my_data_b8_b15 %>% select(1,b10) %>% filter_at(vars(-Integer_Value), all_vars(.== 0)) %>% select(1)
filter_qflags_qc_b11 <- my_data_b8_b15 %>% select(1,b11) %>% filter_at(vars(-Integer_Value), all_vars(.== 0)) %>% select(1)
filter_qflags_qc_b12 <- my_data_b8_b15 %>% select(1,b12) %>% filter_at(vars(-Integer_Value), all_vars(.== 0)) %>% select(1)
filter_qflags_qc_b13 <- my_data_b8_b15 %>% select(1,b13) %>% filter_at(vars(-Integer_Value), all_vars(.== 0)) %>% select(1)
filter_qflags_qc_b14 <- my_data_b8_b15 %>% select(1,b14) %>% filter_at(vars(-Integer_Value), all_vars(.== 0)) %>% select(1)
filter_qflags_qc_b15 <- my_data_b8_b15 %>% select(1,b15) %>% filter_at(vars(-Integer_Value), all_vars(.== 0)) %>% select(1)
filter_qflags_qc_b16 <- my_data_b16 %>% select(1,b16) %>% filter_at(vars(-Integer_Value), all_vars(.== 0)) %>% select(1)


Note: Using an external vector in selections is ambiguous.
ℹ Use `all_of(b8)` instead of `b8` to silence this message.
ℹ See <https://tidyselect.r-lib.org/reference/faq-external-vector.html>.
This message is displayed once per session.
Note: Using an external vector in selections is ambiguous.
ℹ Use `all_of(b9)` instead of `b9` to silence this message.
ℹ See <https://tidyselect.r-lib.org/reference/faq-external-vector.html>.
This message is displayed once per session.
Note: Using an external vector in selections is ambiguous.
ℹ Use `all_of(b10)` instead of `b10` to silence this message.
ℹ See <https://tidyselect.r-lib.org/reference/faq-external-vector.html>.
This message is displayed once per session.
Note: Using an external vector in selections is ambiguous.
ℹ Use `all_of(b11)` instead of `b11` to silence this message.
ℹ See <https://tidyselect.r-lib.org/reference/faq-external-vector.html>.
This message is displayed once per session.
Note: Using an external vector in selections is ambi

In [17]:
%%R
#  copy data into new columns and replace it with NA if quality check failed
modocga$b08 <- modocga$sur_refl_b08
modocga$b08[!modocga$QC_b8_15_1km %in% filter_qflags_qc_b08$Integer_Value] <- NA
modocga$b09 <- modocga$sur_refl_b09
modocga$b09[!modocga$QC_b8_15_1km %in% filter_qflags_qc_b09$Integer_Value] <- NA
modocga$b10 <- modocga$sur_refl_b10
modocga$b10[!modocga$QC_b8_15_1km %in% filter_qflags_qc_b10$Integer_Value] <- NA
modocga$b11 <- modocga$sur_refl_b11
modocga$b11[!modocga$QC_b8_15_1km %in% filter_qflags_qc_b11$Integer_Value] <- NA
modocga$b12 <- modocga$sur_refl_b12
modocga$b12[!modocga$QC_b8_15_1km %in% filter_qflags_qc_b12$Integer_Value] <- NA
modocga$b13 <- modocga$sur_refl_b13
modocga$b13[!modocga$QC_b8_15_1km %in% filter_qflags_qc_b13$Integer_Value] <- NA
modocga$b14 <- modocga$sur_refl_b14
modocga$b14[!modocga$QC_b8_15_1km %in% filter_qflags_qc_b14$Integer_Value] <- NA
modocga$b15 <- modocga$sur_refl_b15
modocga$b15[!modocga$QC_b8_15_1km %in% filter_qflags_qc_b15$Integer_Value] <- NA
modocga$b16 <- modocga$sur_refl_b16
modocga$b16[!modocga$QC_b16_1km %in% filter_qflags_qc_b16$Integer_Value] <- NA



In [18]:
%%R
# adds a flag column if quality check is passed (TRUE)
modocga$qc_flag_b08 <- modocga$QC_b8_15_1km %in% filter_qflags_qc_b08$Integer_Value
modocga$qc_flag_b09 <- modocga$QC_b8_15_1km %in% filter_qflags_qc_b09$Integer_Value
modocga$qc_flag_b10 <- modocga$QC_b8_15_1km %in% filter_qflags_qc_b10$Integer_Value
modocga$qc_flag_b11 <- modocga$QC_b8_15_1km %in% filter_qflags_qc_b11$Integer_Value
modocga$qc_flag_b12 <- modocga$QC_b8_15_1km %in% filter_qflags_qc_b12$Integer_Value
modocga$qc_flag_b13 <- modocga$QC_b8_15_1km %in% filter_qflags_qc_b13$Integer_Value
modocga$qc_flag_b14 <- modocga$QC_b8_15_1km %in% filter_qflags_qc_b14$Integer_Value
modocga$qc_flag_b15 <- modocga$QC_b8_15_1km %in% filter_qflags_qc_b15$Integer_Value
modocga$qc_flag_b16 <- modocga$QC_b16_1km %in% filter_qflags_qc_b16$Integer_Value

## Land Surface Temperatures

In [19]:
%%R
df_lst_terra <- data %>% 
    dplyr::select(YY, MM, DD, sites,LST_Day_1km_terra, QC_Day_terra) %>% 
    mutate(date = ymd(paste(as.character(YY), as.character(MM), as.character(DD), sep="-")))
df_lst_aqua <- data %>% 
    dplyr::select(YY, MM, DD, sites,LST_Day_1km_aqua, QC_Day_aqua) %>% 
    mutate(date = ymd(paste(as.character(YY), as.character(MM), as.character(DD), sep="-")))


In [20]:
%%R
# https://lpdaac.usgs.gov/documents/118/MOD11_User_Guide_V6.pdf
qflags_lst_terra <- sort(unique(df_lst_terra$QC_Day_terra)) # 8 bits
qflags_lst_aqua <- sort(unique(df_lst_aqua$QC_Day_aqua)) # 8 bits


In [21]:
%%R
# create dataframes in which the flags are read into
QC_lst_terra <- data.frame(Integer_Value = qflags_lst_terra, 
                        Bit7 = NA,  Bit6 = NA,  Bit5 = NA,  Bit4 = NA,  Bit3 = NA,  Bit2 = NA,  Bit1 = NA,
                        Bit0 = NA)
QC_lst_aqua <- data.frame(Integer_Value = qflags_lst_aqua, 
                        Bit7 = NA,  Bit6 = NA,  Bit5 = NA,  Bit4 = NA,  Bit3 = NA,  Bit2 = NA,  Bit1 = NA,
                        Bit0 = NA)

In [22]:
%%R
# read the integers of the qflags into the relevant columns of QC_lst
r <- 0
for (i in QC_lst_terra$Integer_Value){
    AsInt <- as.integer(intToBits(i)) # reads in LSB first 
    QC_lst_terra[r+1,2:9]<- AsInt[8:1]
    r <- r+1
}
r <- 0
for (i in QC_lst_aqua$Integer_Value){
    AsInt <- as.integer(intToBits(i)) # reads in LSB first 
    QC_lst_aqua[r+1,2:9]<- AsInt[8:1] 
    r <- r+1
}

In [23]:
%%R
qc_lst_aqua <- as_tibble(QC_lst_aqua)
# columns with the bits of interest for QC_Day of MYD11A1
# starts with LSB
# https://lpdaac.usgs.gov/documents/118/MOD11_User_Guide_V6.pdf
overall <- c(9:8)                          # Bit 0, 1: 00=LST produced, good quality, not necessary to examine more detailed QA 
                                           #           01=LST produced, other quality, recommend examination of more detailed QA
                                           #           10=LST not produced due to cloud effects 
                                           #           11=LST not produced primarily due to reasons other than cloud
good_quality <- c(7:6)                     # Bit 3, 2: 00=good data quality 
                                           #           01=other quality data
                                           #           10=TBD
                                           #           11=TBD
emissitivity_error <- c(5:4)               # Bit 5, 4: 00=average emissivity error <= 0.01 
                                           #           01=average emissivity error <= 0.02
                                           #           10=average emissivity error <= 0.04
                                           #           11=average emissivity error > 0.04
lst_error <- c(3:2)                        # Bit 7, 6: 00=average LST error <= 1K 
                                           #           01=average LST error <= 2K 
                                           #           10=average LST error <= 3K
                                           #           11=average LST error > 3K

# select valid QC_Day: only of the quality control indicates good data quality will the data be accepted
qc_overall            <- qc_lst_aqua %>% select(1,all_of(overall)) %>% filter_at(vars(-Integer_Value), all_vars(.== 0)) %>% select(1)
qc_good_quality       <- qc_lst_aqua %>% select(1,good_quality) %>% filter_at(vars(-Integer_Value), all_vars(.== 0)) %>% select(1)
qc_emissitivity_error <- qc_lst_aqua %>% select(1,emissitivity_error) %>% filter_at(vars(-Integer_Value), all_vars(.== 0)) %>% select(1)
qc_lst_error          <- qc_lst_aqua %>% select(1,lst_error) %>% filter_at(vars(-Integer_Value), all_vars(.== 0)) %>% select(1)
qc_accepted_aqua      <- rbind(qc_overall,qc_good_quality)

Note: Using an external vector in selections is ambiguous.
ℹ Use `all_of(good_quality)` instead of `good_quality` to silence this message.
ℹ See <https://tidyselect.r-lib.org/reference/faq-external-vector.html>.
This message is displayed once per session.
Note: Using an external vector in selections is ambiguous.
ℹ Use `all_of(emissitivity_error)` instead of `emissitivity_error` to silence this message.
ℹ See <https://tidyselect.r-lib.org/reference/faq-external-vector.html>.
This message is displayed once per session.
Note: Using an external vector in selections is ambiguous.
ℹ Use `all_of(lst_error)` instead of `lst_error` to silence this message.
ℹ See <https://tidyselect.r-lib.org/reference/faq-external-vector.html>.
This message is displayed once per session.


In [24]:
%%R
qc_lst_terra <- as_tibble(QC_lst_terra)
# columns with the bits of interest for QC_Day of MOD11A1
# starts with LSB
# https://lpdaac.usgs.gov/documents/118/MOD11_User_Guide_V6.pdf
overall <- c(9:8)                          # Bit 0, 1: 00=LST produced, good quality, not necessary to examine more detailed QA 
                                           #           01=LST produced, other quality, recommend examination of more detailed QA
                                           #           10=LST not produced due to cloud effects 
                                           #           11=LST not produced primarily due to reasons other than cloud
good_quality <- c(7:6)                     # Bit 3, 2: 00=good data quality 
                                           #           01=other quality data
                                           #           10=TBD
                                           #           11=TBD
emissitivity_error <- c(5:4)               # Bit 5, 4: 00=also
                                           #           01=average emissivity error <= 0.02
                                           #           10=average emissivity error <= 0.04
                                           #           11=average emissivity error > 0.04
lst_error <- c(3:2)                        # Bit 7, 6: 00=average LST error <= 1K 
                                           #           01=average LST error <= 2K 
                                           #           10=average LST error <= 3K
                                           #           11=average LST error > 3K

# select valid QC_Day: only of the quality control indicates good data quality will the data be accepted
qc_overall            <- qc_lst_aqua %>% select(1,all_of(overall)) %>% filter_at(vars(-Integer_Value), all_vars(.== 0)) %>% select(1)
qc_good_quality       <- qc_lst_aqua %>% select(1,good_quality) %>% filter_at(vars(-Integer_Value), all_vars(.== 0)) %>% select(1)
qc_emissitivity_error <- qc_lst_aqua %>% select(1,emissitivity_error) %>% filter_at(vars(-Integer_Value), all_vars(.== 0)) %>% select(1)
qc_lst_error          <- qc_lst_aqua %>% select(1,lst_error) %>% filter_at(vars(-Integer_Value), all_vars(.== 0)) %>% select(1)
qc_accepted_terra     <- rbind(qc_overall,qc_good_quality)


In [25]:
%%R
#  copy data into new columns and replace it with NA if quality check failed
df_lst_terra$lst_terra <- df_lst_terra$LST_Day_1km_terra
df_lst_terra$lst_terra[!df_lst_terra$QC_Day_terra %in% qc_accepted_terra$Integer_Value] <- NA
df_lst_aqua$lst_aqua <- df_lst_aqua$LST_Day_1km_aqua
df_lst_aqua$lst_aqua[!df_lst_aqua$QC_Day_aqua %in% qc_accepted_aqua$Integer_Value] <- NA

## Merge MOD09GA, MODOCGA, MOD11A1 and MYD11A1

In [26]:
%%R
filter_data <- merge(mod09, modocga, by=c("YY","MM","DD","sites", "date"),all = T) 
data_refl_lst_aqua <- merge(filter_data, df_lst_aqua, by=c("YY","MM","DD","sites", "date"),all = T)
data_refl_lst <- merge(data_refl_lst_aqua, df_lst_terra, by=c("YY","MM","DD","sites", "date"),all = T)


<span style="color:red">**To do:**</span> Adapt stateQA if there should be requirements at the state_1km geolocation flag

In [27]:
%%R
stateQA = FALSE
# Set state_1km geolocation flag
# https://lpdaac.usgs.gov/documents/306/MOD09_User_Guide_V6.pdfTable 13
qflags_state  <- sort(unique(data_refl_lst$state_1km))# 16 bit
QC_Data_state <- data.frame(Integer_Value = qflags_state, 
                        Bit15 = NA, Bit14 = NA, Bit13 = NA, Bit12 = NA, Bit11 = NA,
                        Bit10 = NA, Bit9 = NA,  Bit8 = NA,  Bit7 = NA,  Bit6 = NA,  Bit5 = NA,  Bit4 = NA,  Bit3 = NA,  Bit2 = NA,  Bit1 = NA,
                        Bit0 = NA)
if(stateQA){
    qflags_state <- sort(unique(data_refl_lst_aqua$state_1km)) # 16 bits

    state_Data <- data.frame(Integer_Value = qflags_state, # In an unsigned representation, these values are the integers between 0 and 65535
                               Bit15 = NA, Bit14 = NA, Bit13 = NA, Bit12 = NA, Bit11 = NA,
                               Bit10 = NA, Bit9 = NA,  Bit8 = NA,  Bit7 = NA,  Bit6 = NA,  
                               Bit5 = NA,  Bit4 = NA,  Bit3 = NA,  Bit2 = NA,  Bit1 = NA,
                               Bit0 = NA)
    r <- 0
    for (i in state_Data$Integer_Value){
        AsInt <- as.integer(intToBits(i)[1:16])
        # AsInt <- as.integer(as.logical(as.binary(i, n=16, logic=F)))
        state_Data[r+1,2:17]<- AsInt[16:1]
        r <- r+1
    }
# !!!!!!!!!!!!!!!!!!!!!!! QC_Data_MODOCGA_b8_b15 ... !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
    # my_data <- as_tibble(QC_Data_state)
    bits <- c(4,5,7:9,15:17) # chosen:      0-1 (cloud state: 00) 
                             #                2 (cloud shadow: 0) 
                             #              8-9 (no or small cirrus: 00 or 01) 
                             #               10 (internal cloud algorithmL: 0)  
                             #               12 (MOD35 snow/ice flag: 0) 
                             #               13 (pixel is adjacent to cloud: 0)
                             # not chosen:  3-5 (land/water flag: 000)  6-7 (Sensor range valid: 00) 11 (internal fire algorithm: 00) 14 (salt pan: 0) 15 (internal snow mask: 0)

    filter_qflags <- state_Data %>% select(1,bits) %>% 
        filter_at(vars(-Integer_Value), all_vars(.== 0)) %>% 
        select(1)

    data_refl_lst_aqua <- data_refl_lst_aqua %>% 
        filter_at(vars(state_1km), all_vars(.%in% filter_qflags$Integer_Value))
}

In [28]:
%%R
# Create Indices: NDVI, EVI, NIRv, CCI, PRI
data_refl_lst$ndvi <- (data_refl_lst$b02  - data_refl_lst$b01 )/(data_refl_lst$b02 + data_refl_lst$b01)
data_refl_lst$evi <-  2.5 * (data_refl_lst$b02 - data_refl_lst$b01) / (data_refl_lst$b02 + 6 * data_refl_lst$b01 - 7.5 * data_refl_lst_aqua$b03 + 0.0001)
data_refl_lst$NIRv <- data_refl_lst$ndvi * (data_refl_lst$sur_refl_b02 * 0.0001)  #NIR
data_refl_lst$cci <- (data_refl_lst$b11 - data_refl_lst$b01)/(data_refl_lst$b11 + data_refl_lst$b01) 
data_refl_lst$pri <- (data_refl_lst$b11 - data_refl_lst$b12)/(data_refl_lst$b11 + data_refl_lst$b12)

In [29]:
%%R
## aggregate to daily mean values (multiple data points given per day in original dataset) and remove outliers
ddf_aggregated <- data_refl_lst %>% group_by( sites, YY, MM, DD, date) %>%
                summarise_at( vars( one_of( c( "lst_aqua", "lst_terra", "b01", "b02", "b03", "b04", "b05", "b06", "b07", "b08", "b09", "b10", "b11", "b12", "b13", "b14", "b15",  "b16","ndvi", "evi", 
                                              "cci", "pri", "NIRv") ) ), list( ~mean(., na.rm=TRUE) ) ) %>%
                mutate_at( vars( one_of( c("lst_aqua", "lst_terra", "b01", "b02", "b03", "b04", "b05", "b06", "b07", "b08", "b09", "b10", "b11", "b12", "b13", "b14", "b15",  "b16",
                                           "ndvi", "evi", "cci", "pri", "NIRv"))), list( ~remove_outliers(., coef=3.0) ) )

In [30]:
%%R
summary(ddf_aggregated)

    sites                 YY             MM               DD       
 Length:241826      Min.   :2000   Min.   : 1.000   Min.   : 1.00  
 Class :character   1st Qu.:2005   1st Qu.: 4.000   1st Qu.: 8.00  
 Mode  :character   Median :2008   Median : 7.000   Median :16.00  
                    Mean   :2008   Mean   : 6.545   Mean   :15.71  
                    3rd Qu.:2011   3rd Qu.:10.000   3rd Qu.:23.00  
                    Max.   :2014   Max.   :12.000   Max.   :31.00  
                                                                   
      date               lst_aqua        lst_terra           b01         
 Min.   :2000-02-24   Min.   :233.9    Min.   :234.6    Min.   :  -47.5  
 1st Qu.:2005-03-19   1st Qu.:286.8    1st Qu.:285.1    1st Qu.:  720.1  
 Median :2008-09-04   Median :295.3    Median :293.9    Median : 2198.5  
 Mean   :2008-05-22   Mean   :295.4    Mean   :293.1    Mean   : 3262.1  
 3rd Qu.:2011-10-07   3rd Qu.:304.6    3rd Qu.:301.9    3rd Qu.: 5671.5  
 Max.   :201

In [31]:
%%R
write_csv(ddf_aggregated, path = "./data/FLUXNET_MODOCGA_MOD09GA_MYD11A1_MOD11A1.csv")